In [ ]:
%load_ext autoreload
%autoreload 2

## We need a better comprehension of Vispy, and OpenGL, to make desired vizualizations properly. http://vispy.org/resources.html

In [ ]:
from vispy import app
from vispy import gloo
import numpy as np
from LeCheapEyeTracker import Client, Server, Stimulation
import time
import cv2

### How to make multiple layers of display?

In [ ]:
vertex = """
    attribute vec2 position;
    attribute vec2 texcoord;
    varying vec2 v_texcoord;
    void main(){
        gl_Position = vec4(position, 0.0, 1.0);
        v_texcoord = texcoord;
    }
"""

fragment = """
    uniform sampler2D texture1;
    uniform sampler2D texture2;
    varying vec2 v_texcoord;
    void main(){
        gl_FragColor = texture2D(texture1, v_texcoord);
        gl_FragColor = texture2D(texture2, v_texcoord);
    }
"""

In [ ]:
class Test(app.Canvas):
    def init(self, timeline):
        self.timeline = timeline
        app.use_app('pyglet')
        app.Canvas.__init__(self, keys='interactive', size=(1280, 720))
        self.fullscreen = True
        self.width, self.height = self.physical_size
        self.prog = gloo.Program(vertex, fragment, count=4)
        self.prog['position'] = [(-1, 1), (-1, 1), (1, -1), (1, 1)]
        self.prog['texcoord'] = [(1, 1), (1, 0), (0, 1), (0, 0)]
        self.img0 = np.zeros((self.height, self.width), 3).astype(np.uint8)
        self.prog['texture1'] = self.img0
        gloo.set_viewport(0, 0, self.width, self.height)
        self._timer = app.Timer('auto', connect=self.on_timer, start=True)
        self.start = time.time()
        self.show()
    
    def on_draw(self, event):
        gloo.clear('black')
        self.prog.draw('triangle_strip')
        
    def on_timer(self, event):
        if time.time()-self.start <self.timeline.max():
            circle = cv2.circle(self.img0.copy(), (1200, 360), 4, (255, 0, 0), 2)
            self.prog['texture1'] = circle
            self.update()
        else :
            self.close()
            
            

In [ ]:
T = 5.
screen = Test(np.linspace(0, T, T*30))
screen.app.run()


In [ ]:
! git commit -am 'greater comprehension of vispy and openGL'

In [ ]:
! git push